# Importando los modulos necesarios

In [ ]:
# Instalación de dependencias
!pip install qkeras keras-tuner

In [ ]:
    import h5py
    import numpy as np
    import json
    import tensorflow as tf
    tf.config.run_functions_eagerly(True)
    tf.data.experimental.enable_debug_mode()
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
    from tensorflow.keras.layers import Input, MaxPooling2D, Flatten, Dropout, Add
    from tensorflow.keras.models import Model
    import matplotlib.pyplot as plt
    from qkeras import *
    import keras_tuner as kt

#Cargando el dataset de escalogramas

In [ ]:


from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/Tesis/Accelerometer_Dataset/accelerometer_BR_256NS_20Scales_cwt_dataset.h5'


In [ ]:
def load_cwt_dataset(file_path='accelerometer_cwt_dataset.h5'):
    """
    Carga el dataset de escalogramas desde un archivo HDF5 y lo prepara para Keras.

    Args:
        file_path: Ruta al archivo HDF5

    Returns:
        dataset: Un diccionario con los datos y metadatos
    """


    with h5py.File(file_path, 'r') as hf:
        # Cargar escalogramas y etiquetas
        x_train = np.array(hf['train']['scalograms'])
        y_train = np.array(hf['train']['labels'])
        x_test = np.array(hf['test']['scalograms'])
        y_test = np.array(hf['test']['labels'])

        # Cargar metadatos
        num_classes = hf['metadata']['num_classes'][()]
        shape = tuple(hf['metadata']['shape'][()])

        # Cargar diccionario de etiquetas
        label_codes_dict = json.loads(hf['metadata'].attrs['label_codes_dict'])

    # Organizar datos como un dataset tipo Keras
    dataset = {
        'train': (x_train, y_train),
        'test': (x_test, y_test),
        'num_classes': num_classes,
        'input_shape': shape,
        'label_codes_dict': label_codes_dict
    }

    print(f"Dataset cargado: {x_train.shape[0]} muestras de entrenamiento, {x_test.shape[0]} muestras de prueba")
    print(f"Forma de cada escalograma: {shape}")
    print(f"Número de clases: {num_classes}")

    return dataset



In [ ]:
# Carga el dataset
dataset = load_cwt_dataset(file_path)

# Obtén las partes del dataset
x_train, y_train = dataset['train']
x_test, y_test = dataset['test']
input_shape = dataset['input_shape']
num_classes = dataset['num_classes']

#Preparar las etiquetas con one-hot encoding

In [ ]:
# Convertir etiquetas a formato one-hot
from tensorflow.keras.utils import to_categorical

# Verificar la forma actual de las etiquetas
print("Forma original de y_train:", y_train.shape)
print("Primeros 5 valores de y_train:", y_train[:5])

# Convertir etiquetas a formato one-hot
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Verificar la nueva forma
print("Nueva forma de y_train:", y_train.shape)
print("Primeros 5 valores de y_train:", y_train[:5])

#Verificando la normalización de los datos

In [ ]:
# Verificar si los datos ya están normalizados
print("Estado actual de los datos:")
print("Valor máximo en x_train:", np.max(x_train))
print("Valor mínimo en x_train:", np.min(x_train))
print("Media de x_train:", np.mean(x_train))
print("Desviación estándar de x_train:", np.std(x_train))

# Visualizar la distribución de valores
plt.figure(figsize=(10, 4))
plt.hist(x_train.flatten(), bins=50)
plt.title('Distribución de valores en x_train')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.show()

#No estan normalizado, a normalizar!!

In [ ]:
# Normalizar los datos de entrada usando el máximo de x_train para ambos conjuntos
max_value = np.max(x_train)  # Usar sólo el conjunto de entrenamiento para calcular el valor

# Convertir a float32 y normalizar
x_train = x_train.astype('float32') / max_value
x_test = x_test.astype('float32') / max_value  # Usar el mismo valor para test

print("Después de normalizar:")
print("Valor máximo en x_train:", np.max(x_train))
print("Valor mínimo en x_train:", np.min(x_train))
print("Valor máximo en x_test:", np.max(x_test))
print("Valor mínimo en x_test:", np.min(x_test))

#Adaptando los datos para el modelo CNN

In [ ]:
print("Forma de x_train:", x_train.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de x_test:", x_test.shape)
print("Forma de y_test:", y_test.shape)
# Adaptar dimensiones para CNN (necesitamos un canal)
# Asumiendo que las dimensiones actuales son (n_samples, scales, time_steps)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

print("Forma final de x_train:", x_train.shape)
print("Forma final de x_test:", x_test.shape)

#Separamos una parte de los datos para la validacion del modelo

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos
x_train_final, x_val, y_train_final, y_val = train_test_split(
    x_train, y_train,
    test_size=0.2,  # 20% para validación
    random_state=42,  # Para reproducibilidad
    stratify=y_train  # Mantener la distribución de clases
)

# Verificar tamaños
print(f"Datos de entrenamiento: {x_train_final.shape[0]} muestras")
print(f"Datos de validación: {x_val.shape[0]} muestras")

# Definiendo los bloques convolucionales de mi modelo

In [ ]:
# Habilitar ejecución eager para TensorFlow
import tensorflow as tf
tf.config.run_functions_eagerly(True)

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, MaxPooling2D, Flatten, Dropout, Add, Activation
import keras_tuner as kt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Usar el adaptador corregido
from bnn_adapter_fixed import binary_activation, binary_conv, binary_dense, my_flat
from bnn_adapter_fixed import l1_batch_norm_mod_conv, l1_batch_norm_mod_dense

def bnn_model_builder(hp):
    """
    Constructor de modelo BNN para búsqueda de arquitectura.
    Combina enfoque de NAS con capas binarias optimizadas para microcontroladores.
    """
    # Hiperparámetros para búsqueda
    conv_stages = hp.Int('conv_stages', min_value=1, max_value=3, step=1)
    initial_filters = hp.Choice('initial_filters', values=[16, 32, 64])
    use_residual = hp.Boolean('use_residual')
    dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.3, step=0.1)

    # Configuración para batch norm
    batch_norm_momentum = hp.Float('batch_norm_momentum',
                                  min_value=0.8, max_value=0.95, step=0.05)

    # Entrada: escalogramas de señales vibroacústicas
    inputs = Input(shape=(x_train_final.shape[1], x_train_final.shape[2], 1))

    # Primera capa - siempre usar mayor precisión en primera capa
    # Usamos una convolución estándar (no binaria) para preservar información de entrada
    # Esto sigue las recomendaciones para STM32 y el paper de Wang
    x = tf.keras.layers.Conv2D(filters=initial_filters,
                               kernel_size=3,
                               padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.BatchNormalization(momentum=batch_norm_momentum)(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Bloques de convolución binarios
    filters = initial_filters

    for i in range(conv_stages):
        # Más bloques en etapas posteriores como en EfficientNetV2
        num_blocks = 1 + i

        for j in range(num_blocks):
            # Guardar entrada para conexión residual
            res_input = x

            # Convolución binaria con backprop personalizada (del paper)
            x = binary_conv(
                nfilters=filters,
                ch_in=int(x.shape[-1]),
                k=3,
                padding='same'
            )(x)

            # MaxPool antes de BatchNorm (recomendación STM32)
            if j == num_blocks - 1 and i < conv_stages - 1:  # Al final de cada etapa
                x = MaxPooling2D(pool_size=(2, 2))(x)

            # BatchNorm L1 modificado (del paper)
            x = l1_batch_norm_mod_conv(
                batch_size=64,  # Podría ser hiperparámetro
                width_in=x.shape[1],
                ch_in=filters,
                momentum=batch_norm_momentum
            )(x)

            # Activación binaria
            x = binary_activation()(x)

            # Conexión residual cuando sea posible
            if use_residual and j > 0 and res_input.shape == x.shape:
                x = Add()([res_input, x])

            # Dropout
            if dropout_rate > 0:
                x = Dropout(dropout_rate)(x)

        # Aumentar filtros para siguiente etapa
        filters *= 2

    # Aplanar
    x = my_flat()(x)

    # Capas densas binarias
    for units in [256, 128]:
        x = binary_dense(
            n_in=int(x.shape[-1]),
            n_out=units
        )(x)
        x = l1_batch_norm_mod_dense(
            batch_size=64,  # Podría ser hiperparámetro
            ch_in=units,
            momentum=batch_norm_momentum
        )(x)
        x = binary_activation()(x)
        if dropout_rate > 0:
            x = Dropout(dropout_rate)(x)

    # Capa de salida - no binaria para mejor precisión
    x = tf.keras.layers.Dense(5)(x)  # 5 clases
    outputs = Activation('softmax')(x)

    model = Model(inputs, outputs)

    # Optimizador personalizado para BNN
    lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

#Función NAS con Keras Tuner

In [ ]:
def bnn_model_builder(hp):
    """
    Constructor de modelo BNN para búsqueda de arquitectura.
    Combina enfoque de NAS con capas binarias optimizadas para microcontroladores.
    """
    # Hiperparámetros para búsqueda
    conv_stages = hp.Int('conv_stages', min_value=1, max_value=3, step=1)
    initial_filters = hp.Choice('initial_filters', values=[16, 32, 64])
    use_residual = hp.Boolean('use_residual')
    dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.3, step=0.1)

    # Configuración para batch norm
    batch_norm_momentum = hp.Float('batch_norm_momentum',
                                  min_value=0.8, max_value=0.95, step=0.05)

    # Entrada: escalogramas de señales vibroacústicas
    inputs = Input(shape=(x_train_final.shape[1], x_train_final.shape[2], 1))

    # Primera capa - siempre usar mayor precisión en primera capa
    # Usamos una convolución estándar (no binaria) para preservar información de entrada
    # Esto sigue las recomendaciones para STM32 y el paper de Wang
    x = tf.keras.layers.Conv2D(filters=initial_filters,
                               kernel_size=3,
                               padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.BatchNormalization(momentum=batch_norm_momentum)(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Bloques de convolución binarios
    filters = initial_filters

    for i in range(conv_stages):
        # Más bloques en etapas posteriores como en EfficientNetV2
        num_blocks = 1 + i

        for j in range(num_blocks):
            # Guardar entrada para conexión residual
            res_input = x

            # Convolución binaria con backprop personalizada (del paper)
            x = binary_conv(
                nfilters=filters,
                ch_in=int(x.shape[-1]),
                k=3,
                padding='same'
            )(x)

            # MaxPool antes de BatchNorm (recomendación STM32)
            if j == num_blocks - 1 and i < conv_stages - 1:  # Al final de cada etapa
                x = MaxPooling2D(pool_size=(2, 2))(x)

            # BatchNorm L1 modificado (del paper)
            x = l1_batch_norm_mod_conv(
                batch_size=64,  # Podría ser hiperparámetro
                width_in=x.shape[1],
                ch_in=filters,
                momentum=batch_norm_momentum
            )(x)

            # Activación binaria
            x = binary_activation()(x)

            # Conexión residual cuando sea posible
            if use_residual and j > 0 and res_input.shape == x.shape:
                x = Add()([res_input, x])

            # Dropout
            if dropout_rate > 0:
                x = Dropout(dropout_rate)(x)

        # Aumentar filtros para siguiente etapa
        filters *= 2

    # Aplanar
    x = my_flat()(x)

    # Capas densas binarias
    for units in [256, 128]:
        x = binary_dense(
            n_in=int(x.shape[-1]),
            n_out=units
        )(x)
        x = l1_batch_norm_mod_dense(
            batch_size=64,  # Podría ser hiperparámetro
            ch_in=units,
            momentum=batch_norm_momentum
        )(x)
        x = binary_activation()(x)
        if dropout_rate > 0:
            x = Dropout(dropout_rate)(x)

    # Capa de salida - no binaria para mejor precisión
    x = tf.keras.layers.Dense(5)(x)  # 5 clases
    outputs = Activation('softmax')(x)

    model = Model(inputs, outputs)

    # Optimizador personalizado para BNN
    lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Función de Estimación de Recursos Mejorada para BNN

In [ ]:
def estimate_bnn_resources(model):
    """
    Estima uso de recursos para un modelo BNN en STM32L432KC.
    Las BNN son mucho más eficientes que los modelos cuantizados estándar.
    """
    # Contar capas binarias
    total_params = model.count_params()
    binary_params = 0

    for layer in model.layers:
        if isinstance(layer, binary_conv) or isinstance(layer, binary_dense):
            # Contar parámetros en capas binarias
            binary_params += layer.count_params()

    # En BNN, cada peso ocupa solo 1 bit en lugar de 8
    effective_param_size = (total_params - binary_params) + (binary_params / 8)

    # Estimar memoria RAM (más preciso para BNN)
    # Las activaciones binarias también usan menos memoria
    activation_memory_kb = 5  # Estimación base

    # Memoria para parámetros
    param_memory_kb = effective_param_size / 8 / 1024

    # Memoria de trabajo
    working_memory_kb = 10

    total_memory_kb = param_memory_kb + activation_memory_kb + working_memory_kb
    flash_usage_kb = effective_param_size / 8 / 1024 + 50  # Código base ~50KB

    return {
        'total_params': total_params,
        'binary_params': binary_params,
        'effective_param_size': effective_param_size,
        'ram_usage_kb': total_memory_kb,
        'flash_usage_kb': flash_usage_kb
    }

# Búsqueda de Arquitectura Multi-Objetivo para BNN

In [ ]:
class BNNTunerMultiObjective(kt.BayesianOptimization):
    def __init__(self, *args,
                 accuracy_weight=0.6,
                 size_weight=0.2,
                 ram_weight=0.2,
                 max_ram_kb=48,
                 **kwargs):
        super().__init__(*args, **kwargs)
        self.accuracy_weight = accuracy_weight
        self.size_weight = size_weight
        self.ram_weight = ram_weight
        self.max_ram_kb = max_ram_kb

    def run_trial(self, trial, *args, **kwargs):
        hp = trial.hyperparameters
        model = self.hypermodel.build(hp)

        # Estimar recursos
        resources = estimate_bnn_resources(model)
        print(f"Modelo con {resources['total_params']:,} parámetros")
        print(f"- Parámetros binarios: {resources['binary_params']:,}")
        print(f"- RAM estimada: {resources['ram_usage_kb']:.1f} KB")
        print(f"- Flash estimado: {resources['flash_usage_kb']:.1f} KB")

        # Entrenar modelo
        results = super().run_trial(trial, *args, **kwargs)

        if results["status"] != kt.engine.trial.TrialStatus.OK:
            # Si hay errores, devolver resultados sin score
            return results

        # Calcular puntuación compuesta
        val_accuracy = results["metrics"]["val_accuracy"]

        # Normalizar puntuaciones (mayor es mejor)
        accuracy_score = val_accuracy
        size_score = max(0, 1.0 - (resources['flash_usage_kb'] / 256))
        ram_score = max(0, 1.0 - (resources['ram_usage_kb'] / self.max_ram_kb))

        # Score final ponderado
        composite_score = (
            self.accuracy_weight * accuracy_score +
            self.size_weight * size_score +
            self.ram_weight * ram_score
        )

        # Mostrar resultados
        print(f"\nModelo evaluado:")
        print(f"- Precisión: {val_accuracy:.4f}")
        print(f"- Score tamaño: {size_score:.4f}")
        print(f"- Score RAM: {ram_score:.4f}")
        print(f"- Puntuación final: {composite_score:.4f}\n")

        # Actualizar score
        results["score"] = composite_score
        return results

# Función Principal para Ejecutar la Búsqueda

In [ ]:
def run_bnn_nas():
    """
    Ejecuta la búsqueda de arquitectura para BNN optimizada para STM32
    """
    print("Iniciando búsqueda de arquitectura para BNN...")

    # Instanciar el tuner personalizado
    tuner = BNNTunerMultiObjective(
        bnn_model_builder,
        objective='val_accuracy',
        max_trials=20,
        directory='nas_results',
        project_name='bnn_escalogramas_stm32',
        accuracy_weight=0.6,
        size_weight=0.2,
        ram_weight=0.2,
        max_ram_kb=48
    )

    # Callbacks para entrenamiento
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                         restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
    ]

    # Realizar la búsqueda
    tuner.search(
        x_train_final, y_train_final,
        validation_data=(x_val, y_val),
        epochs=30,  # Menos épocas para la búsqueda inicial
        batch_size=64,
        callbacks=callbacks
    )

    # Obtener mejor modelo
    best_model = tuner.get_best_models(1)[0]
    best_hps = tuner.get_best_hyperparameters(1)[0]

    # Mostrar configuración óptima
    print("\n=== MEJOR ARQUITECTURA BNN ===")
    print(f"Etapas convolucionales: {best_hps.get('conv_stages')}")
    print(f"Filtros iniciales: {best_hps.get('initial_filters')}")
    print(f"Usar residual: {best_hps.get('use_residual')}")
    print(f"Dropout: {best_hps.get('dropout_rate')}")
    print(f"Momentum BatchNorm: {best_hps.get('batch_norm_momentum')}")
    print(f"Tasa de aprendizaje: {best_hps.get('learning_rate')}")

    # Estimar recursos del mejor modelo
    resources = estimate_bnn_resources(best_model)
    print(f"Parámetros totales: {resources['total_params']:,}")
    print(f"Parámetros binarios: {resources['binary_params']:,}")
    print(f"Tamaño efectivo: {resources['effective_param_size']:,}")
    print(f"RAM estimada: {resources['ram_usage_kb']:.1f} KB")
    print(f"Flash estimado: {resources['flash_usage_kb']:.1f} KB")

    return best_model, best_hps

# Entrenamiento Final del Mejor Modelo

In [ ]:
def train_best_bnn_model(best_model):
    """
    Entrena el mejor modelo BNN encontrado con parámetros óptimos
    """
    # Optimizador con schedule de tasa de aprendizaje (como en Binary.py)
    initial_lr = 0.01
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    best_model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Callbacks para entrenamiento avanzado
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            patience=15, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(
            factor=0.5, patience=10, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(
            'best_bnn_model.h5', save_best_only=True)
    ]

    # Normalización específica para BNN (-1 a 1 en lugar de 0 a 1)
    # Las BNN funcionan mejor con datos centrados en 0
    x_train_final_bnn = 2.0 * x_train_final - 1.0
    x_val_bnn = 2.0 * x_val - 1.0
    x_test_bnn = 2.0 * x_test - 1.0

    # Entrenamiento
    print("\nEntrenando modelo BNN final...")
    history = best_model.fit(
        x_train_final_bnn, y_train_final,
        validation_data=(x_val_bnn, y_val),
        epochs=100,
        batch_size=64,
        callbacks=callbacks
    )

    # Evaluación
    test_loss, test_acc = best_model.evaluate(x_test_bnn, y_test)
    print(f"Precisión en conjunto de prueba: {test_acc:.4f}")

    # Visualizar resultados
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Precisión del modelo BNN')
    plt.ylabel('Precisión')
    plt.xlabel('Época')
    plt.legend(['Entrenamiento', 'Validación'], loc='lower right')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Pérdida del modelo BNN')
    plt.ylabel('Pérdida')
    plt.xlabel('Época')
    plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
    plt.show()

    return best_model, history

Función para Exportación y Preparación para STM32

In [ ]:
def export_bnn_for_stm32(model):
    """
    Prepara el modelo BNN para su implementación en STM32
    """
    # Guardar modelo
    model.save('bnn_model_for_stm32.h5')

    # Resumir parámetros binarios para implementación manual si es necesario
    binary_weights = {}

    for i, layer in enumerate(model.layers):
        if isinstance(layer, binary_conv) or isinstance(layer, binary_dense):
            weights = layer.get_weights()
            # Los pesos ya están binarizados para inferencia
            binary_weights[f"layer_{i}"] = weights

    # Guardar pesos binarios como archivo numpy para referencia
    np.save('bnn_binary_weights.npy', binary_weights)

    print("Modelo guardado para STM32.")
    print("Para usar con STM32Cube.AI o implementación manual.")

# Ejecución

In [ ]:
# Verificar que los datos estén divididos correctamente
if not 'x_val' in locals():
    print("Dividiendo datos para validación...")
    x_train_final, x_val, y_train_final, y_val = train_test_split(
        x_train, y_train,
        test_size=0.2,
        random_state=42
    )

# Ejecutar NAS para BNN
best_bnn_model, best_hps = run_bnn_nas()

# Entrenar mejor modelo
final_model, history = train_best_bnn_model(best_bnn_model)

# Exportar para STM32
export_bnn_for_stm32(final_model)